<a href="https://colab.research.google.com/github/alina-chak/HTML-SESSSION2/blob/master/_Improved_SBD_Gygli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#OPENCV VERSION FOR USING SIFT
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17


In [ ]:
!git clone https://github.com/oladeha2/shot_boudary_detector.git

Cloning into 'shot_boudary_detector'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [ ]:
%cd /content/
!ls

/content
sample_data  shot_boudary_detector


In [ ]:
%cd /content/shot_boudary_detector
!ls

/content/shot_boudary_detector
get_transition_frames_cpu.py		TestVideo.py
get_transition_frames_gpu.py		transition_network.py
README.md				utilities.py
shot_boundary_detector_even_distrib.pt	video_processing.py
snippet.py


In [ ]:
!python image_save.py /content/v1.mp4

decomposing video to frames this may take a while  for large videos :) .....
final size:  (64, 64)
frame decomposition complete !!! 


In [ ]:
import sys
import shutil

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from snippet import getSnippet
from math import floor
from transition_network import TransitionCNN
from utilities import normalize_frame, print_shape
import pandas as pd
import os
import time
from TestVideo import TestVideo, return_start_and_end
from moviepy.editor import VideoFileClip
from video_processing import six_four_crop_video
from PIL import Image

# command line arguments --> file name, video_file_name, gpu or cpu 


# first decompose the video to frames
# place the video to be detected into the directory 

video = sys.argv[1]
pred_text_file_name = sys.argv[2]


text_file = 'frames.txt'

print('decomposing video to frames this may take a while  for large videos :) .....')
frames_path = 'video_frames/'
os.makedirs('video_frames/', exist_ok=True)
os.makedirs('predictions/', exist_ok=True)

vid = VideoFileClip(video)
vid = six_four_crop_video(vid)

frames = [frame for frame in vid.iter_frames()]

f = open(text_file, 'w+')

for j, frame in enumerate(frames):
        frame_path = frames_path + 'frame_' + str(j+1) + '.png'
        im = Image.fromarray(frame)
        im.save(frame_path)            
        f.write(frame_path + '\n')    

print('frame decomposition complete !!! ')


In [ ]:
#CODE FOR GETTING NO OF FRAMES
import cv2
cap= cv2.VideoCapture('/content/v1.mp4')

totalframecount= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("The total number of frames in this video is ", totalframecount)


The total number of frames in this video is  14541


In [ ]:
#CODE FOR EXTRACTING VIDEO FRAMES
import cv2

# Opens the Video file
cap= cv2.VideoCapture('/content/v1.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('kang'+str(i)+'.jpg',frame)
    i+=1

cap.release()
cv2.destroyAllWindows()

In [ ]:
#CHANGES IN CODE FOR FALSE PREDICTION REMOVAL
import sys
import shutil

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from snippet import getSnippet
from math import floor
from transition_network import TransitionCNN
from utilities import normalize_frame, print_shape
import pandas as pd
import os
import time
from TestVideo import TestVideo, return_start_and_end
from moviepy.editor import VideoFileClip
from video_processing import six_four_crop_video
from PIL import Image
import cv2 
#import matplotlib.pyplot as plt
#%matplotlib inline

# command line arguments --> file name, video_file_name, gpu or cpu 


# first decompose the video to frames
# place the video to be detected into the directory 

video = sys.argv[1]
pred_text_file_name = sys.argv[2]


text_file = 'frames.txt'

print('decomposing video to frames this may take a while  for large videos :) .....')
frames_path = 'video_frames/'
os.makedirs('video_frames/', exist_ok=True)
os.makedirs('predictions/', exist_ok=True)

vid = VideoFileClip(video)
vid = six_four_crop_video(vid)

frames = [frame for frame in vid.iter_frames()]

f = open(text_file, 'w+')

for j, frame in enumerate(frames):
        frame_path = frames_path + 'frame_' + str(j+1) + '.png'
        im = Image.fromarray(frame)
        im.save(frame_path)            
        f.write(frame_path + '\n')    

print('frame decomposition complete !!! ')

device = 'cuda'

#load model
model = TransitionCNN()
model.load_state_dict(torch.load('shot_boundary_detector_even_distrib.pt'))
model.to(device)

prediction_text_file = 'predictions/' + pred_text_file_name 

pred_file = open(prediction_text_file, 'w+')

print('computing predictions for video', video, '...................' )

test_video = TestVideo('frames.txt', sample_size=100, overlap=9)
test_loader = DataLoader(test_video, batch_size=1, num_workers=1)

video_indexes = []
vals = np.arange(test_video.get_line_number())
length = len(test_video)

for val in range(length):
    s,e = return_start_and_end(val)
    video_indexes.append(vals[s:e])

for indx, batch in enumerate(test_loader):
        batch.to(device)
        batch = batch.type('torch.cuda.FloatTensor')
        predictions = model(batch)
        predictions = predictions.argmax(dim=1).cpu().numpy()
        for idx, prediction_set in enumerate(predictions):
            for i, prediction in enumerate(prediction_set):
                if prediction[0][0] == 0:
                    frame_index = video_indexes[indx][i+5]
                    #print(frame_index)
                    frame_path0 = frames_path + 'frame_' + str(int(frame_index)-1) + '.png'
                    frame_path = frames_path + 'frame_' + str(frame_index) + '.png'
                    #print(frame_path0)
                    # read images
                    img1 = cv2.imread(frame_path0)  
                    img2 = cv2.imread(frame_path) 

                    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
                    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

                    #sift
                    sift = cv2.xfeatures2d.SIFT_create()

                    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
                    keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
                    a = max()
                    print(len(keypoints_1))
                    #feature matching
                    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

                    matches = bf.match(descriptors_1,descriptors_2)
                    #print(len(matches))
                    #matches = sorted(matches, key = lambda x:x.distance)
                    #print(len(matches))
                    print(len(matches)/len(keypoints_1))
                    




                    
                    pred_file.write(str(frame_index) + '\n')
pred_file.close()

# delete files used for process
os.remove('frames.txt')
shutil.rmtree('video_frames/')

print('Predictions complete !!!')
print('Frames that are part of shot boundaries are listed in file the directory path predictions/' + pred_text_file_name)














In [ ]:
!python get_transition_frames_gpu.py /content/v1.mp4 pred1.txt

In [ ]:
#using SURF features
!python get_transition_frames_gpu.py /content/v3.mp4 pred2.txt

In [ ]:
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline
fr0 = '/content/messi_3.jfif'
fr1 = '/content/messi_4.jfif'
# read images
img1 = cv2.imread(fr0)  
img2 = cv2.imread(fr1) 

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#sift
sift = cv2.xfeatures2d.SIFT_create()

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
print(len(keypoints_1))
print(len(keypoints_2))
#feature matching
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)

print(len(matches))

print(len(matches)/len(keypoints_1))

#img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:50], img2, flags=2)
#plt.imshow(img3),plt.show()

In [ ]:
#CHANGES IN CODE FOR FALSE PREDICTION REMOVAL KEYPOINT TESTING
import sys
import shutil

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from snippet import getSnippet
from math import floor
from transition_network import TransitionCNN
from utilities import normalize_frame, print_shape
import pandas as pd
import os
import time
from TestVideo import TestVideo, return_start_and_end
from moviepy.editor import VideoFileClip
from video_processing import six_four_crop_video
from PIL import Image
import cv2 
#import matplotlib.pyplot as plt
#%matplotlib inline

# command line arguments --> file name, video_file_name, gpu or cpu 


# first decompose the video to frames
# place the video to be detected into the directory 

video = sys.argv[1]
pred_text_file_name = sys.argv[2]

frames_path = '/content/shot_boudary_detector/video_frames/'


text_file = 'frames.txt'


device = 'cuda'

#load model
model = TransitionCNN()
model.load_state_dict(torch.load('shot_boundary_detector_even_distrib.pt'))
model.to(device)

prediction_text_file = 'predictions/' + pred_text_file_name 

pred_file = open(prediction_text_file, 'w+')

print('computing predictions for video', video, '...................' )

test_video = TestVideo('frames.txt', sample_size=100, overlap=9)
test_loader = DataLoader(test_video, batch_size=1, num_workers=1)

video_indexes = []
vals = np.arange(test_video.get_line_number())
length = len(test_video)

for val in range(length):
    s,e = return_start_and_end(val)
    video_indexes.append(vals[s:e])

for indx, batch in enumerate(test_loader):
        batch.to(device)
        batch = batch.type('torch.cuda.FloatTensor')
        predictions = model(batch)
        predictions = predictions.argmax(dim=1).cpu().numpy()
        for idx, prediction_set in enumerate(predictions):
            for i, prediction in enumerate(prediction_set):
                if prediction[0][0] == 0:
                    frame_index = video_indexes[indx][i+5]
                    #print(frame_index)
                    frame_path0 = frames_path + 'frame_' + str(int(frame_index)-1) + '.png'
                    frame_path = frames_path + 'frame_' + str(frame_index) + '.png'
                    #print(frame_path0)
                    # read images
                    img1 = cv2.imread(frame_path0)  
                    img2 = cv2.imread(frame_path) 

                    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
                    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

                    #sift
                    sift = cv2.xfeatures2d.SIFT_create()
                    #BRISK
                    brisk = cv2.BRISK_create()

                    keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
                    keypoints_2, descriptors_2 = brisk.detectAndCompute(img2,None)

                    print(len(keypoints_2))
                    #feature matching
                    #bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

                    #matches = bf.match(descriptors_1,descriptors_2)
                    #print(len(matches))
                    #matches = sorted(matches, key = lambda x:x.distance)
                    #print(len(matches))
                    #print(len(matches)/len(keypoints_1))
                    




                    
                    pred_file.write(str(frame_index) + '\n')
pred_file.close()

# delete files used for process
os.remove('frames.txt')
shutil.rmtree('video_frames/')

print('Predictions complete !!!')
print('Frames that are part of shot boundaries are listed in file the directory path predictions/' + pred_text_file_name)














In [ ]:
import cv2
img1 = cv2.imread('/content/messi_3.jfif')  
img2 = cv2.imread('/content/messi_4.jfif') 

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

                    #sift
sift = cv2.xfeatures2d.SIFT_create()

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)

print(len(keypoints_1))
                    #feature matching
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
print(len(matches))
                    #matches = sorted(matches, key = lambda x:x.distance)
                    #print(len(matches))
                    #print(len(matches)/len(keypoints_1))
                    

In [ ]:
#CHANGES IN CODE FOR FALSE PREDICTION REMOVAL
import sys
import shutil

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np
from snippet import getSnippet
from math import floor
from transition_network import TransitionCNN
from utilities import normalize_frame, print_shape
import pandas as pd
import os
import time
from TestVideo import TestVideo, return_start_and_end
from moviepy.editor import VideoFileClip
from video_processing import six_four_crop_video
from PIL import Image
import cv2 
#import matplotlib.pyplot as plt
#%matplotlib inline

# command line arguments --> file name, video_file_name, gpu or cpu 


# first decompose the video to frames
# place the video to be detected into the directory 

video = sys.argv[1]
pred_text_file_name = sys.argv[2]


text_file = 'frames.txt'

print('decomposing video to frames this may take a while  for large videos :) .....')
frames_path = 'video_frames/'
os.makedirs('video_frames/', exist_ok=True)
os.makedirs('predictions/', exist_ok=True)

vid = VideoFileClip(video)
vid = six_four_crop_video(vid)

frames = [frame for frame in vid.iter_frames()]

f = open(text_file, 'w+')

for j, frame in enumerate(frames):
        frame_path = frames_path + 'frame_' + str(j+1) + '.png'
        im = Image.fromarray(frame)
        im.save(frame_path)            
        f.write(frame_path + '\n')    

print('frame decomposition complete !!! ')

device = 'cuda'

#load model
model = TransitionCNN()
model.load_state_dict(torch.load('shot_boundary_detector_even_distrib.pt'))
model.to(device)

prediction_text_file = 'predictions/' + pred_text_file_name 

pred_file = open(prediction_text_file, 'w+')

print('computing predictions for video', video, '...................' )

test_video = TestVideo('frames.txt', sample_size=100, overlap=9)
test_loader = DataLoader(test_video, batch_size=1, num_workers=1)

video_indexes = []
vals = np.arange(test_video.get_line_number())
length = len(test_video)

for val in range(length):
    s,e = return_start_and_end(val)
    video_indexes.append(vals[s:e])

for indx, batch in enumerate(test_loader):
        batch.to(device)
        batch = batch.type('torch.cuda.FloatTensor')
        predictions = model(batch)
        predictions = predictions.argmax(dim=1).cpu().numpy()
        for idx, prediction_set in enumerate(predictions):
            for i, prediction in enumerate(prediction_set):
                if prediction[0][0] == 0:
                    frame_index = video_indexes[indx][i+5]
                    #print(frame_index)
                    frame_path0 = frames_path + 'frame_' + str(int(frame_index)-1) + '.png'
                    frame_path = frames_path + 'frame_' + str(frame_index) + '.png'
                    #print(frame_path0)
                    # read images
                    img1 = cv2.imread(frame_path0)  
                    img2 = cv2.imread(frame_path) 

                    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
                    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

                    #sift
                    #sift = cv2.xfeatures2d.SIFT_create()
                    surf = cv2.xfeatures2d.SURF_create()

                    #keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
                    keypoints_2, descriptors_2 = surf.detectAndCompute(img2,None)
                    
                    print(len(keypoints_2))
                    #feature matching
                    #bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

                    #matches = bf.match(descriptors_1,descriptors_2)
                    #print(len(matches))
                    #matches = sorted(matches, key = lambda x:x.distance)
                    #print(len(matches))
                    #print(len(matches)/len(keypoints_1))
                    if (len(keypoints_2) != 0):               
                      pred_file.write(str(frame_index) + '\n')
                    
pred_file.close()

# delete files used for process
os.remove('frames.txt')
shutil.rmtree('video_frames/')

print('Predictions complete !!!')
print('Frames that are part of shot boundaries are listed in file the directory path predictions/' + pred_text_file_name)














In [ ]:
#BRISK KEYPOINTS
!python get_transition_frames_gpu.py /content/v10.mp4 pred2.txt